# Combine into One DataFrame the Generalizability Metrics from Experiments on All Datasets for One Model

## Papermill Parameters

In [ ]:
PARAM_DEFINITIONS_PATH = "../definitions"
PARAM_OUTPUT_PATH = "../outputs"

###### Switch these parameters ######
# parameters for base setup
PARAM_COVER_SIZE_WEIGHT = 0
PARAM_CLASS_BALANCE_WEIGHT = 0
PARAM_ENABLED_GENERALIZATION_AWARENESS = False
PARAM_ENABLED_SIGNIFICANCE_FILTERING = False

# parameters for setup of our full method
# PARAM_COVER_SIZE_WEIGHT = 1
# PARAM_CLASS_BALANCE_WEIGHT = 1
# PARAM_ENABLED_GENERALIZATION_AWARENESS = True
# PARAM_ENABLED_SIGNIFICANCE_FILTERING = True

PARAM_INPUT_EXPERIMENTS = {
    "sklearn_naive_bayes_GaussianNB": ["exp-18", "exp-19"] + [f"exp-{i}" for i in range(38, 43+1)],
    "xgboost_XGBClassifier": [f"exp-{i}" for i in range(44, 51+1)],
    "sklearn_linear_model_LogisticRegression": [f"exp-{i}" for i in range(52, 59+1)],
    "sklearn_ensemble_RandomForestClassifier": [f"exp-{i}" for i in range(85, 92+1)],
    "sklearn_neural_network_MLPClassifier": [f"exp-{i}" for i in range(93, 100+1)]}
PARAM_INPUT_STAGES = {"exp-18": "stage-14", "exp-19": "stage-14"} | {f"exp-{i}": f"stage-{14}" for i in range(38, 59+1)} | {f"exp-{i}": f"stage-{14}" for i in range(85, 100+1)}
PARAM_INPUT_FILES = {f"exp-{i}": [[f"generalizability_metrics_{PARAM_COVER_SIZE_WEIGHT}_{PARAM_CLASS_BALANCE_WEIGHT}_False_average_ranking_loss_{PARAM_ENABLED_GENERALIZATION_AWARENESS}.csv", "ARL"],
                                [f"generalizability_metrics_{PARAM_COVER_SIZE_WEIGHT}_{PARAM_CLASS_BALANCE_WEIGHT}_False_prc_auc_score_{PARAM_ENABLED_GENERALIZATION_AWARENESS}.csv", "PR AUC"],
                                [f"generalizability_metrics_{PARAM_COVER_SIZE_WEIGHT}_{PARAM_CLASS_BALANCE_WEIGHT}_False_sklearn.metrics.roc_auc_score_{PARAM_ENABLED_GENERALIZATION_AWARENESS}.csv", "ROC AUC"]] for i in [18, 19]} \
    | {f"exp-{i}": [[f"generalizability_metrics_{PARAM_COVER_SIZE_WEIGHT}_{PARAM_CLASS_BALANCE_WEIGHT}_average_ranking_loss_{PARAM_ENABLED_GENERALIZATION_AWARENESS}.csv", "ARL"],
                    [f"generalizability_metrics_{PARAM_COVER_SIZE_WEIGHT}_{PARAM_CLASS_BALANCE_WEIGHT}_prc_auc_score_{PARAM_ENABLED_GENERALIZATION_AWARENESS}.csv", "PR AUC"],
                    [f"generalizability_metrics_{PARAM_COVER_SIZE_WEIGHT}_{PARAM_CLASS_BALANCE_WEIGHT}_sklearn.metrics.roc_auc_score_{PARAM_ENABLED_GENERALIZATION_AWARENESS}.csv", "ROC AUC"]] for i in list(range(38, 59+1)) + list(range(85, 100+1))}

PARAM_MODEL = "xgboost_XGBClassifier"

## Select Input Experiment for Requested Model

In [ ]:
input_experiments = PARAM_INPUT_EXPERIMENTS[PARAM_MODEL]

print(PARAM_INPUT_EXPERIMENTS)
print(PARAM_INPUT_STAGES)
print(PARAM_INPUT_FILES)

## Load and Combine Results

In [ ]:
import pandas as pd
import yaml
import os

metrics = set()
stage_dicts = []

for exp in input_experiments:
    with open(f"{PARAM_DEFINITIONS_PATH}/{exp}/stage-00/global_params.yaml") as global_params_file:
        global_params = yaml.load(global_params_file, Loader=yaml.FullLoader)

    exp_base_row = {"experiment": exp, "dataset": global_params["PARAM_DATASET_NAME"], "model_type": global_params["PARAM_MODEL_TYPE"]}

    stage = PARAM_INPUT_STAGES[exp]
    for input_filename, performance_measure_name in PARAM_INPUT_FILES[exp]:
        try:
            stage_row_part = {key: value[0] for key, value in pd.read_csv(f"{PARAM_OUTPUT_PATH}/{exp}/{stage}/{input_filename}").to_dict().items()}
            metrics.update(stage_row_part.keys())

            print(f"{PARAM_OUTPUT_PATH}/{exp}/{stage}/{input_filename} values: {stage_row_part}")
        except FileNotFoundError:
            stage_row_part = {}

            print(f"{PARAM_OUTPUT_PATH}/{exp}/{stage}/{input_filename} not found")

        stage_row_part |= {"performance_measure": performance_measure_name}
        stage_row = exp_base_row | stage_row_part
        stage_dicts.append(stage_row)

all_outputs_df = pd.DataFrame(stage_dicts)

meta_reports_path = f"{PARAM_OUTPUT_PATH}/meta_reports/generalizability_metrics"
os.makedirs(meta_reports_path, exist_ok=True)

all_outputs_df.to_csv(f"{meta_reports_path}/generalizability_metrics.csv", index=False)

## Turn into LaTeX Table Code

In [ ]:
import numpy as np


def combine_num_subgroups(row, col1, col2, col3=None):
    val1 = format(row[col1], "{0:.0f}")
    val2 = format(row[col2], "{0:.0f}")

    if col3 is not None:
        val3 = format(row[col3], "{0:.0f}")
        return f"{val1}/{val2}/{val3}"
    else:
        return f"{val1}/{val2}"


def format(x, fstring):
    if isinstance(x, str):
        return x

    if np.isnan(x):
        return "-"

    return fstring.format(x)


def makecell(str):
    return r"\makecell{" + str + r"}"


def rotatebox(str):
    return r"\rotatebox{90}{" + str + r"}"


num_subgroups_column_name = None
mean_exceptionality_column_name = None
mean_pairwise_iou_column_name = None
cover_size_column_name = None
ncr_column_name = None

if PARAM_ENABLED_SIGNIFICANCE_FILTERING:
    mean_exceptionality_column_name = "top-k filtered mean subtractive search exceptionality"
    mean_pairwise_iou_column_name = "top-k filtered mean pairwise IoU"
    cover_size_column_name = "top-k filtered mean search cover size"
    ncr_column_name = "top-k filtered mean search NCR"
    intermediate_num_subgroups_column_name = "number of filtered subgroups"
    final_num_subgroups_column_name = "number of top-k filtered subgroups"

    num_subgroups_column_name = "combined number of subgroups"
    all_outputs_df[num_subgroups_column_name] = all_outputs_df.loc[:, [final_num_subgroups_column_name, intermediate_num_subgroups_column_name, "number of full subgroups"]] \
                                                .apply(lambda x: combine_num_subgroups(x, final_num_subgroups_column_name, intermediate_num_subgroups_column_name, "number of full subgroups"), axis=1)
    num_subgroups_column_header_name = r"Result Set Size\\(Top-5/\\Significant/\\Original)"
else:
    mean_exceptionality_column_name = "top-k full mean subtractive search exceptionality"
    mean_pairwise_iou_column_name = "top-k full mean pairwise IoU"
    cover_size_column_name = "top-k full mean search cover size"
    ncr_column_name = "top-k full mean search NCR"
    top_k_num_subgroups_column_name = "number of top-k full subgroups"
    filtered_top_k_num_subgroups_column_name = "number of filtered top-k subgroups"

    num_subgroups_column_name = "combined number of subgroups"
    all_outputs_df[filtered_top_k_num_subgroups_column_name] = ["(not implemented, count manually)"]*len(all_outputs_df)
    all_outputs_df[num_subgroups_column_name] = all_outputs_df.loc[:, [filtered_top_k_num_subgroups_column_name, top_k_num_subgroups_column_name]] \
                                                .apply(lambda x: combine_num_subgroups(x, filtered_top_k_num_subgroups_column_name, top_k_num_subgroups_column_name), axis=1)
    num_subgroups_column_header_name = r"Result Set Size\\(Significant/\\Top-5)"

latex_df = all_outputs_df.loc[:, ["dataset", "performance_measure", mean_exceptionality_column_name, mean_pairwise_iou_column_name, cover_size_column_name, ncr_column_name, num_subgroups_column_name]]

latex_df[mean_exceptionality_column_name] = latex_df[mean_exceptionality_column_name].map(lambda x: format(x, "{0:.2f}"))
latex_df[mean_pairwise_iou_column_name] = latex_df[mean_pairwise_iou_column_name].map(lambda x: format(x, "{0:.2f}"))
latex_df[cover_size_column_name] = latex_df[cover_size_column_name].map(lambda x: format(x, "{0:.0f}"))
latex_df[ncr_column_name] = latex_df[ncr_column_name].map(lambda x: format(x, "{0:.2f}"))
latex_df[num_subgroups_column_name] = latex_df[num_subgroups_column_name].map(lambda x: format(x, "{0:.0f}"))

latex_df.rename(columns={
    "dataset": "Dataset", 
    "performance_measure": makecell(r"Interestingness\\Measure"),
    mean_exceptionality_column_name: rotatebox(makecell(r"Mean\\Exceptionality")),
    mean_pairwise_iou_column_name: rotatebox(makecell(r"Mean\\Pairwise IoU")),
    cover_size_column_name: rotatebox(makecell(r"Mean\\Cover Size")),
    ncr_column_name: rotatebox(makecell(r"Mean NCR")),
    num_subgroups_column_name: rotatebox(makecell(num_subgroups_column_header_name)),
    }, inplace=True)

datasets_line_breaks_dict = {
   "UCI Credit Approval": r"UCI Credit Approval",
   "UCI Breast Cancer Wisconsin": r"UCI Breast\\Cancer Wisconsin",
   "Statlog (German Credit Data)": r"Statlog (German\\Credit Data)",
   "UCI Mushroom": r"UCI Mushroom",
   "UCI Credit Card Clients": r"UCI Credit\\Card Clients",
   "UCI Bank Marketing": r"UCI Bank\\Marketing",
   "OpenML Adult": r"OpenML Adult",
   "UCI Census-Income (KDD)": r"UCI Census-Income\\(KDD)",
}

multiindex_columns = ["Dataset", r"\makecell{Interestingness\\Measure}"] 
multiindex = pd.MultiIndex.from_frame(latex_df[multiindex_columns])
latex_df.drop(multiindex_columns, axis=1, inplace=True)
latex_df.set_index(multiindex, inplace=True)

latex_df.sort_index(inplace=True)
latex_df.sort_index(level=0, ascending=False, inplace=True, key=lambda x: [list(datasets_line_breaks_dict.keys()).index(idx) for idx in x])

datasets = latex_df.index.levels[0]

latex_df.style.to_latex(f"{meta_reports_path}/generalizability_metrics_{PARAM_MODEL}_{PARAM_COVER_SIZE_WEIGHT}_{PARAM_CLASS_BALANCE_WEIGHT}_{PARAM_ENABLED_GENERALIZATION_AWARENESS}_{PARAM_ENABLED_SIGNIFICANCE_FILTERING}.tex",
                        column_format="llrrrrl",
                        hrules=True,
                        clines="skip-last;data")